In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import scipy.stats as stats
from scipy.interpolate import interp1d
from scipy.optimize import minimize,least_squares,dual_annealing,differential_evolution
from xtquant import xtdatacenter as xtdc
from xtquant import xtdata
from datetime import datetime

xtdc.set_token('4065054877ce5724155dbc5bcba200381ce5eb35')
xtdc.init()

xtquant250516.1.1已经发布,前往 http://dict.thinktrader.net/nativeApi/download_xtquant.html 查看更新说明



In [30]:
def get_market_data(codes: list, period: str, start_time: str, end_time: str, count: int) -> dict:
    """获取市场数据"""
    for code in codes:
        xtdata.subscribe_quote(code, period, start_time, end_time, count, callback=None)
        xtdata.download_history_data(code, period, start_time, end_time)
    market_data = xtdata.get_market_data_ex([], codes, period, start_time, end_time, count=count, 
                                          dividend_type='none', fill_data=False)
    for code in codes:
        if market_data[code].empty:
            print(f"{code} 数据获取失败！")
            break
    
    return market_data

In [2]:
def parse_option_code(option_code):
    """解析期权代码"""
    code = option_code.split('.')[0]
    if '-' in code:
        parts = code.split('-')
        if len(parts) == 3:
            underlying = parts[0]
            option_type = 'call' if parts[1].upper() == 'C' else 'put'
            strike = int(parts[2])
        else:
            import traceback
            traceback.print_exc()
    else:
        call_pos = code.find('C')
        put_pos = code.find('P')
        if call_pos != -1:
            option_type = 'call'
            underlying = code[:call_pos]
            strike = int(code[call_pos + 1:])
        elif put_pos != -1:
            option_type = 'put'
            underlying = code[:put_pos]
            strike = int(code[put_pos + 1:])
        else:
            import traceback
            traceback.print_exc()
    return {
        'option_type': option_type,
        'underlying': underlying,
        'strike': strike,
        'exchange': option_code.split('.')[-1] if '.' in option_code else None
    }

In [ ]:
xtdata.download_history_contracts()
future_list = ['jd2508.DF', 'jd2509.DF']


***** xtdata连接成功 *****
服务信息: {'tag': 'xtquant', 'version': '1.0'}
服务地址: 127.0.0.1:58609
数据路径: c:\Users\HP\Desktop\Heston\data\datadir
设置xtdata.enable_hello = False可隐藏此消息



In [33]:
def get_option_data(future_list: list, date_str: str) -> pd.DataFrame:
    """获取某个时间节点的横截面期权数据"""
    option_list = []
    option_dict = {}
    
    for future_code in future_list:
        options = xtdata.get_option_list(undl_code = future_code,dedate = '20250701',opttype = '',isavailavle= True)
        if options:  # 确保options不为空
            option_list.extend(options)
    
    option_data = get_market_data(codes = option_list, period = '1d', start_time = date_str, end_time = date_str, count = -1) 

    for option_code in option_list:
        try:
            # 获取期权的结算价格
            if option_code in option_data and not option_data[option_code].empty:
                settlement_price = option_data[option_code]['settelementPrice'].iloc[-1]  # 获取最新的结算价
                
                expire_date_str = xtdata.get_option_detail_data(option_code)['ExpireDate']
                expire_date = datetime.strptime(expire_date_str, '%Y%m%d')
                dt = datetime.strptime(date_str, '%Y%m%d')
                delta_t = (expire_date - dt).days / 365.0

                future_code = xtdata.get_option_detail_data(option_code)['OptUndlCodeFull']
                future_price = get_market_data(codes=[future_code], period='1d', start_time=date_str, end_time=date_str, count=-1)[future_code]['settelementPrice'].iloc[-1]

                # 解析期权代码
                parsed_info = parse_option_code(option_code)
                
                # 创建该期权的字典
                option_dict[option_code] = {
                    'option_price': settlement_price,
                    'future_price': future_price,
                    'option_type': parsed_info['option_type'],
                    'strike': parsed_info['strike'],
                    'time_to_expire': delta_t
                }
                
            else:
                print(f"期权代码 {option_code} 的数据不存在或为空")
                
        except Exception as e:
            print(f"处理期权代码 {option_code} 时出错: {e}")

    # 将字典转换为DataFrame
    df = pd.DataFrame.from_dict(option_dict, orient='index')
    df.reset_index(inplace=True)
    df.rename(columns={'index': 'option_code'}, inplace=True)

    return df

In [35]:
get_option_data(future_list = future_list, date_str = '20250701')

,option_code,option_price,future_price,option_type,strike,time_to_expire
0,jd2508-C-3150.DF,418.0,3568.0,call,3150,0.041096
1,jd2508-C-3200.DF,369.0,3568.0,call,3200,0.041096
2,jd2508-C-3250.DF,320.0,3568.0,call,3250,0.041096
3,jd2508-C-3300.DF,273.0,3568.0,call,3300,0.041096
4,jd2508-C-3350.DF,227.5,3568.0,call,3350,0.041096
...,...,...,...,...,...,...
83,jd2509-P-4100.DF,425.0,3690.0,put,4100,0.131507
84,jd2509-P-4200.DF,517.5,3690.0,put,4200,0.131507
85,jd2509-P-4300.DF,613.5,3690.0,put,4300,0.131507
86,jd2509-P-4400.DF,711.5,3690.0,put,4400,0.131507


In [ ]:
option_dict = {}

for option_code in option_list:
    try:
        # 获取期权的结算价格
        if option_code in option_data and not option_data[option_code].empty:
            settlement_price = option_data[option_code]['settelementPrice'].iloc[-1]  # 获取最新的结算价
            
            expire_date_str = xtdata.get_option_detail_data(option_code)['ExpireDate']
            expire_date = datetime.strptime(expire_date_str, '%Y%m%d')
            today = datetime.now()
            delta_t = (expire_date - today).days / 365.0

            future_code = xtdata.get_option_detail_data('jd2508-C-3150.DF')['OptUndlCodeFull']
            future_price = get_single_market_data(code=future_code, period='1d', start_time='20250630', end_time='20250630', count=-1)

            # 解析期权代码
            parsed_info = parse_option_code(option_code)
            
            # 创建该期权的字典
            option_dict[option_code] = {
                'price': settlement_price,
                'option_type': parsed_info['option_type'],
                'strike': parsed_info['strike'],
                'time_to_expire': delta_t
            }
            
        else:
            print(f"期权代码 {option_code} 的数据不存在或为空")
            
    except Exception as e:
        print(f"处理期权代码 {option_code} 时出错: {e}")

In [7]:
df = pd.DataFrame.from_dict(option_dict, orient='index')
df.reset_index(inplace=True)
df.rename(columns={'index': 'option_code'}, inplace=True)
df.to_csv('jd.csv', index=False, encoding='utf-8-sig')

In [8]:
option_list

['jd2508-C-3150.DF',
 'jd2508-C-3200.DF',
 'jd2508-C-3250.DF',
 'jd2508-C-3300.DF',
 'jd2508-C-3350.DF',
 'jd2508-C-3400.DF',
 'jd2508-C-3450.DF',
 'jd2508-C-3500.DF',
 'jd2508-C-3550.DF',
 'jd2508-C-3600.DF',
 'jd2508-C-3650.DF',
 'jd2508-C-3700.DF',
 'jd2508-C-3750.DF',
 'jd2508-C-3800.DF',
 'jd2508-C-3850.DF',
 'jd2508-C-3900.DF',
 'jd2508-C-3950.DF',
 'jd2508-C-4000.DF',
 'jd2508-C-4100.DF',
 'jd2508-C-4200.DF',
 'jd2508-C-4300.DF',
 'jd2508-C-4400.DF',
 'jd2508-C-4600.DF',
 'jd2508-P-3150.DF',
 'jd2508-P-3200.DF',
 'jd2508-P-3250.DF',
 'jd2508-P-3300.DF',
 'jd2508-P-3350.DF',
 'jd2508-P-3400.DF',
 'jd2508-P-3450.DF',
 'jd2508-P-3500.DF',
 'jd2508-P-3550.DF',
 'jd2508-P-3600.DF',
 'jd2508-P-3650.DF',
 'jd2508-P-3700.DF',
 'jd2508-P-3750.DF',
 'jd2508-P-3800.DF',
 'jd2508-P-3850.DF',
 'jd2508-P-3900.DF',
 'jd2508-P-3950.DF',
 'jd2508-P-4000.DF',
 'jd2508-P-4100.DF',
 'jd2508-P-4200.DF',
 'jd2508-P-4300.DF',
 'jd2508-P-4400.DF',
 'jd2508-P-4600.DF',
 'jd2509-C-3250.DF',
 'jd2509-C-33

'jd2508.DF'

{'jd2509.DF': Empty DataFrame
 Columns: [time, open, high, low, close, volume, amount, settelementPrice, openInterest, preClose, suspendFlag]
 Index: []}

In [14]:
xtdata.download_history_data('jd2509.DF', '1d', '20250630', '20250630') 

{'rb2401.SF': Empty DataFrame
 Columns: [time, open, high, low, close, volume, amount, settelementPrice, openInterest, preClose, suspendFlag]
 Index: []}

,time,open,high,low,close,volume,amount,settelementPrice,openInterest,preClose,suspendFlag
20250630,1751212800000,3654.0,3694.0,3644.0,3689.0,74788,2.746994e+09,3673.0,165823,3673.0,0
